This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]
ResampleType = ttk.ResampleImage[ImageType]

#IMPORTANT: This assumed the data has been brain-stripped and resampled into isotropic spacing!!!
reader = ReaderType.New(FileName="MRA-Iso.mha")
reader.Update()
im1iso = reader.GetOutput()
reader = ReaderType.New(FileName="MRA-Brain-VesselEnhanced.mha")
reader.Update()
im1BrainVess = reader.GetOutput()

In [3]:
view(im1BrainVess)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [4]:
imMath = ttk.ImageMath[ImageType,ImageType].New()
imMath.SetInput(im1Brain)
imMath.Blur(1.0)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 10
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
    indx.reverse()
    seedCoord[:][i] = im1Brain.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[-15.9078321  -22.9855871    6.66245282]
 [ -2.60888692  -2.65183671  -1.04874474]
 [-13.41554095 -12.13042089  -0.80657562]
 [-27.82812546 -24.4508738   18.37093524]
 [-42.88031365 -15.51437618  22.45765273]
 [-12.70866419  -3.12293465  14.04063522]
 [ 14.92985178  10.67370493  17.04517199]
 [-18.34659598   8.88268063  16.25303385]
 [-23.14955895  -7.12923827  -6.79776154]
 [-53.24560825 -10.77376713  26.09933615]]


In [5]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinCurvature(0.001)
vSeg.SetRadiusInObjectSpace( 1 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [-15.9078321  -22.9855871    6.66245282]
**** Processing seed 1 : [-2.60888692 -2.65183671 -1.04874474]
**** Processing seed 2 : [-13.41554095 -12.13042089  -0.80657562]
**** Processing seed 3 : [-27.82812546 -24.4508738   18.37093524]
**** Processing seed 4 : [-42.88031365 -15.51437618  22.45765273]
**** Processing seed 5 : [-12.70866419  -3.12293465  14.04063522]
**** Processing seed 6 : [14.92985178 10.67370493 17.04517199]
**** Processing seed 7 : [-18.34659598   8.88268063  16.25303385]
**** Processing seed 8 : [-23.14955895  -7.12923827  -6.79776154]
**** Processing seed 9 : [-53.24560825 -10.77376713  26.09933615]


In [6]:
view(tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [7]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [8]:
view(fgMask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [9]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.SetInput( im1iso )
enhancer.AddInput( im2iso )
enhancer.AddInput( im3iso )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 128 )
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.3333,1,2.5])
enhancer.Update()
enhancer.ClassifyImages()

In [10]:
#view(enhancer.GetOutput())
view(enhancer.GetClassProbabilityImage(0))
#imMath = ttk.ImageMath[ImageType,ImageType].New(Input = segmenter.GetClassProbabilityImage(0))
#imMath.AddImages( segmenter.GetClassProbabilityImage(1), 1, -1 )
#view(imMath.GetOutput())

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [11]:
#itk.imwrite( enhancer.GetClassProbabilityImage(0), "MRA-VesselProb.mha", compression=True)
#itk.imwrite( enhancer.GetClassProbabilityImage(1), "MRA-NotVesselProb.mha", compression=True)

In [42]:
im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
itk.imwrite( im1vess, "MRA-VesselEnhanced.mha", compression=True)

brainMask = itk.imread( "MRMask-Brain.mha", itk.F )
im1BrainVess = itk.MultiplyImageFilter(Input1 = im1vess, Input2=brainMask)
itk.imwrite( im1BrainVess, "MRA-Brain-VesselEnhanced.mha", compression=True)